<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A1/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install sentence-transformers==2.2.2
!pip install bitsandbytes
!pip install datasets
!pip install -qU langchain-huggingface
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 810.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-yflgcg0p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-

In [2]:

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import os
import random
import string
import warnings
from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

warnings.filterwarnings('ignore')

### Vector Database creation

In [3]:
#embedding model
embedding_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs={'device':'cuda'},
    encode_kwargs={'device': 'cuda', 'batch_size': 16}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_model,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)

In [14]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="Sir Keir Starmer became Prime Minister on 5 July 2024. The Prime Minister is the leader of His Majesty’s Government and is ultimately responsible for the policy and decisions of the government.",
    metadata={"source": "website"},
    id=1,
)

document_2 = Document(
    page_content="Question: One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid? Note that the pyramid is equilateral and solid.\nAnswer: As there are 11 balls along one side, it means that there are 11 layers of balls. The top most layer has 1 ball. The second layer has 3 (1+2) balls. The third layer has 6 (1+2+3) balls. The fourth layer has 10 (1+2+3+4) balls. The fifth layer has 15 (1+2+3+4+5) balls. Similarly, there are 21, 28, 36, 45, 55 and 66 balls in the remaining layers. Hence, the total number of balls is = 1 + 3 + 6 + 10 + 15 + 21 + 28 + 36 + 45 + 55 + 66 = 286 balls",
    metadata={"source": "website"},
    id=2,
)

document_3 = Document(
    page_content="Question: When a number is divided by 5, the remainder is 3. What is the remainder when twice the number is divided by 5? Answer: 1",
    metadata={"source": "website"},
    id=3,
)

fact_check1 = Document(
    page_content="Muhammad Yunus, 2006 Nobel Peace Prize winner, has been appointed chief adviser to Bangladesh’s interim government following Prime Minister Sheikh Hasina’s resignation amid protests. Yunus, born in 1940, is an economist known for pioneering microcredit through Grameen Bank, founded in 1983. The bank has loaned over $34 billion to nearly 10 million people, with a 97% repayment rate, inspiring global microfinance initiatives. Yunus’s appointment aims to address unrest over government job quotas.",
    metadata={"source": "news"},
    id=4,
)

fact_check2 = Document(
    page_content="Indian wrestler Vinesh Phogat, 29, announced her retirement after being disqualified from the Olympics for being 100 grams overweight before her gold medal bout in the 50 kg category. She appealed to the Court of Arbitration for Sports (CAS) to be awarded a joint silver medal, citing severe dehydration from extreme weight-cutting measures. Phogat, a three-time Olympian in different weight classes, became the first international wrestler to defeat reigning champion Yui Susaki and the first Indian female wrestler to reach an Olympic final in 2024. She is a multiple Commonwealth Games gold medalist and has won two World Wrestling Championship bronze medals.",
    metadata={"source": "news"},
    id=5,
)

fact_check3 = Document(
    page_content="Archaeologists have discovered a terracotta pipeline during the 10th phase of excavation at Keeladi, a Sangam Era site in Tamil Nadu. This find highlights advanced water management practices from over 2,600 years ago. Keeladi, located near Madurai, has revealed over 20,000 artifacts, suggesting the Sangam Era began around 600 BCE, earlier than previously thought. Additional discoveries include rice husks from 3,200 years ago and evidence of early iron technology.",
    metadata={"source": "news"},
    id=6,
)

fact_check4 = Document(
    page_content="The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave, held on August 1-2, 2024, in New Delhi, was inaugurated by Director General Rakesh Pal. Themed ‘Towards an Inclusive Approach,’ the event included sessions on holistic living, stress management, and leadership. It provided a platform for officers to share ideas with decision-makers, focusing on inclusive growth, technology integration, and the principles of Aatmanirbhar Bharat and good governance.",
    metadata={"source": "news"},
    id=7,
)

fact_check5 = Document(
    page_content="It is given that the two digits of the required number are prime numbers i.e. 2, 3, 5 or 7. Note that 1 is neither prime nor composite. Also, the third digit is the multiplication of the first two digits. Thus, hundreds digit and units digit must be either 2 or 3 i.e. 2_2, 2_3, 3_2 or 3_3 which means that there are four possible numbers - 242, 263, 362 and 393. Now, it is also given that - the difference between its reverse and it is 99. So 263 and 362 satisfy this condition. Hence, the sum of the three digits is 11 in each case.",
    metadata={"source": "website"},
    id=8,
)

documents = [
    document_1,
    document_2,
    document_3,
    fact_check1,
    fact_check2,
    fact_check3,
    fact_check4,
    fact_check5,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['8df489bd-d202-4083-81b1-2713b46e4359',
 'c07034ea-fe70-4b69-a69c-341ef7e8a446',
 '533300c2-0260-416a-90b9-920a2b0436bf',
 '6ec4c60e-cd63-45c3-ab48-4be499a32f3f',
 '17b83828-4df5-4c9e-94bc-d5d81cdeebf3',
 'a95fd9c6-5794-4fda-bd9b-e5e9c30efc08',
 '8bb54b95-03eb-4579-929f-1928cbb4b7f7',
 '96cd898d-a8a7-4a71-ab99-06f7ff3efe93']

In [6]:
results = vector_store.similarity_search_with_score(
    "where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.384350] The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave, held on August 1-2, 2024, in New Delhi, was inaugurated by Director General Rakesh Pal. Themed ‘Towards an Inclusive Approach,’ the event included sessions on holistic living, stress management, and leadership. It provided a platform for officers to share ideas with decision-makers, focusing on inclusive growth, technology integration, and the principles of Aatmanirbhar Bharat and good governance. [{'source': 'news'}]


In [7]:
results = vector_store.similarity_search_with_score(
    "who has been appointed as the chief adviser to Bangladesh’s interim government?", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.003037] Muhammad Yunus, 2006 Nobel Peace Prize winner, has been appointed chief adviser to Bangladesh’s interim government following Prime Minister Sheikh Hasina’s resignation amid protests. Yunus, born in 1940, is an economist known for pioneering microcredit through Grameen Bank, founded in 1983. The bank has loaned over $34 billion to nearly 10 million people, with a 97% repayment rate, inspiring global microfinance initiatives. Yunus’s appointment aims to address unrest over government job quotas. [{'source': 'news'}]


In [8]:
results = vector_store.similarity_search(
    "which Indian wrestler has announced her retirement after being disqualified from the Olympics?", k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Indian wrestler Vinesh Phogat, 29, announced her retirement after being disqualified from the Olympics for being 100 grams overweight before her gold medal bout in the 50 kg category. She appealed to the Court of Arbitration for Sports (CAS) to be awarded a joint silver medal, citing severe dehydration from extreme weight-cutting measures. Phogat, a three-time Olympian in different weight classes, became the first international wrestler to defeat reigning champion Yui Susaki and the first Indian female wrestler to reach an Olympic final in 2024. She is a multiple Commonwealth Games gold medalist and has won two World Wrestling Championship bronze medals. [{'source': 'news'}]


In [9]:
results = vector_store.similarity_search(
    "archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?", k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Archaeologists have discovered a terracotta pipeline during the 10th phase of excavation at Keeladi, a Sangam Era site in Tamil Nadu. This find highlights advanced water management practices from over 2,600 years ago. Keeladi, located near Madurai, has revealed over 20,000 artifacts, suggesting the Sangam Era began around 600 BCE, earlier than previously thought. Additional discoveries include rice husks from 3,200 years ago and evidence of early iron technology. [{'source': 'news'}]


## Llama3.1

In [10]:
from huggingface_hub import login
from google.colab import userdata

# Login with your API token
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
import transformers
import torch

class llama:

  def __init__(self, vector_store):
    self.model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    self.pipeline = transformers.pipeline(
        "text-generation",
        model=self.model_id,
        device_map="auto",
        model_kwargs={
          "torch_dtype": torch.bfloat16,
          "quantization_config": {"load_in_4bit": True},
          "low_cpu_mem_usage": True,
        },

    )

    self.vector_store = vector_store

  def generate(self, prompt):

    messages = [
        {"role": "user", "content": prompt},
    ]

    terminators = [
        self.pipeline.tokenizer.eos_token_id,
        self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = self.pipeline(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.3,
        top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]["content"]

  def generate_with_rag(self, prompt):
    res = self.vector_store.similarity_search(prompt, k=1)
    rag_prompt = f"Knowledge {res[0].page_content} Question: {prompt}"
    return self.generate(rag_prompt)


In [12]:
llama_model = llama(vector_store)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

### Self Inconsistency 1

In [18]:
llama_model.generate("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to archaeological findings, the terracotta pipeline was unearthed at the Pattipulam site in Tamil Nadu, which dates back to the Sangam Era (3rd century BCE to 3rd century CE).'

In [19]:
llama_model.generate("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to reports, archaeologists have unearthed a terracotta pipeline at the ancient Sangam Era site of Keezhadi in Tamil Nadu, India.'

In [20]:
llama_model.generate_with_rag("Archaeologists have unearthed a terracotta pipeline at which Sangam Era site in Tamil Nadu?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to the text, the answer is Keeladi.'

### Self Inconsistency 2

In [21]:
llama_model.generate("How many balls are there in the whole pyramid if one side of the bottom layer of a triangular pyramid has 11 balls? Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'1'

In [22]:
llama_model.generate("One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid?  Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'66'

In [23]:
llama_model.generate_with_rag("One side of the bottom layer of a triangular pyramid has 11 balls. How many are there in the whole pyramid? Just give answer no explanation.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'286'

### Self Inconsistency 3

In [15]:
llama_model.generate("A 3 digit number is such that its tens digit is equal to the product of the other two digits which are prime. Also, the difference between its reverse and itself is 99. What is the sum of the three digits? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'147'

In [16]:
llama_model.generate("When a number has three digits, its tens digit is the same as the product of the other two prime digits. Furthermore, there is a 99-difference between its opposite and itself. The three numbers add up to what? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'171'

In [17]:
llama_model.generate_with_rag("A 3 digit number is such that its tens digit is equal to the product of the other two digits which are prime. Also, the difference between its reverse and itself is 99. What is the sum of the three digits? Give answer no explanation")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'11'

### Fact Checking 1

In [24]:
llama_model.generate("who has been appointed as the chief adviser to Bangladesh’s interim government?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"According to various sources, Fakhruddin Ahmed, a renowned economist and former governor of Bangladesh Bank, was appointed as the Chief Adviser to Bangladesh's interim government in 2007."

In [25]:
llama_model.generate_with_rag("who has been appointed as the chief adviser to Bangladesh’s interim government?")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"According to the passage, Muhammad Yunus, the 2006 Nobel Peace Prize winner, has been appointed as the chief adviser to Bangladesh's interim government."

### Fact Checking 2

In [32]:
llama_model.generate("Which Indian wrestler has announced her retirement after being disqualified from the Paris Olympics 2024?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'There is no Indian wrestler who has announced her retirement after being disqualified from the Paris Olympics 2024. The Paris Olympics 2024 have not yet taken place, and the qualification process is still ongoing.'

In [33]:
llama_model.generate_with_rag("Which Indian wrestler has announced her retirement after being disqualified from the Paris Olympics 2024?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'According to the passage, Vinesh Phogat, a 29-year-old Indian wrestler, has announced her retirement after being disqualified from the Paris Olympics 2024.'

### Fact Checking 3

In [28]:
llama_model.generate("Where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'The 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave was inaugurated in Kochi, Kerala.'

In [29]:
llama_model.generate_with_rag("Where was the 7th Indian Coast Guard (ICG) Subordinate Officers’ Conclave inaugurated?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


"The 7th Indian Coast Guard (ICG) Subordinate Officers' Conclave was inaugurated in New Delhi."

### OpenHathi Prompting and Setup

In [ ]:
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'sarvamai/OpenHathi-7B-Hi-v0.1-Base',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 1
)

In [ ]:
inputs = tokenizer("I'll bet the video game is alot more fun than the film.\Sentiment:", return_tensors="pt")
print(inputs)
input_ids = inputs["input_ids"].to('cuda')

{'input_ids': tensor([[    1,   306, 29915,   645,  1010,   278,  4863,  3748,   338,   394,
           327,   901,  2090,  1135,   278,  2706,  7790, 29903,   296,  2073,
         29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

I'll bet the video game is alot more fun than the film.\Sentiment: negative


In [ ]:
import pandas as pd

splits = {'train': 'pair-class/train-00000-of-00001.parquet', 'dev': 'pair-class/dev-00000-of-00001.parquet', 'test': 'pair-class/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/sentence-transformers/all-nli/" + splits["train"])

df.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
